# UIUC Reboot Capstone Project

## Overview
You work for a regional soft drink distribution center that has access to invoice data from convenience stores (invoices.csv), product data (items.csv), and customer reviews (the files in reviews/).

You would like to examine the data for a few features, then plug them into a recommendation engine to see if you can improve sales.

### Loading Data

In [2]:
import pandas as pd
import glob as glob
import zipfile
import numpy as np               # to get unique values from list using numpy.unique 


In [56]:
# Read in files here

print("Compiling Product Reviews:\t\t", end='')

zf = zipfile.ZipFile('reviews.zip')
datReviews = [filename for filename in zf.namelist() if filename.endswith(".dat")]
print(len(datReviews))


Compiling Product Reviews:		50000


In [3]:
# Basic dataframe info

# Invoice description 

In [3]:
invoice = pd.read_csv('invoice.csv')
print(invoice.info())
print()
print(invoice.columns)
print()
print("Column Names:")
for col in invoice.columns:
    print(col)
    print()
invoice

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930508 entries, 0 to 930507
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Invoice_id    930508 non-null  object
 1   Date          930508 non-null  object
 2   Item_id       930508 non-null  int64 
 3   Vendor_id     930508 non-null  int64 
 4   Vendor_Name   930508 non-null  object
 5   Store_id      930508 non-null  int64 
 6   Store_Name    930508 non-null  object
 7   Address       930508 non-null  object
 8   City_Name     930508 non-null  object
 9   Zip_Code      930508 non-null  int64 
 10  County_id     930508 non-null  int64 
 11  County_Name   930508 non-null  object
 12  Bottles_Sold  930508 non-null  int64 
dtypes: int64(6), object(7)
memory usage: 92.3+ MB
None

Index(['Invoice_id', 'Date', 'Item_id', 'Vendor_id', 'Vendor_Name', 'Store_id',
       'Store_Name', 'Address', 'City_Name', 'Zip_Code', 'County_id',
       'County_Name', 'Bottles_Sold']

,Invoice_id,Date,Item_id,Vendor_id,Vendor_Name,Store_id,Store_Name,Address,City_Name,Zip_Code,County_id,County_Name,Bottles_Sold
0,INV-00013400001,2016-08-29,35918,297,Katou Company,3882,Kwik Shop #579 / Davenport,2805 TELEGRAPH RD,DAVENPORT,52802,82,Scott,1
1,INV-00013400002,2016-08-29,23828,297,Katou Company,3882,Kwik Shop #579 / Davenport,2805 TELEGRAPH RD,DAVENPORT,52802,82,Scott,1
2,INV-00013400003,2016-08-29,36908,300,Katsuragi Company,3882,Kwik Shop #579 / Davenport,2805 TELEGRAPH RD,DAVENPORT,52802,82,Scott,1
3,INV-00013400004,2016-08-29,34359,35,Archer Inc.,3882,Kwik Shop #579 / Davenport,2805 TELEGRAPH RD,DAVENPORT,52802,82,Scott,8
4,INV-00013400005,2016-08-29,36903,300,Katsuragi Company,3882,Kwik Shop #579 / Davenport,2805 TELEGRAPH RD,DAVENPORT,52802,82,Scott,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
930503,S34122500013,2016-08-25,68031,260,Inuyasha Brands,4694,Rina Mart LLC / Davenport,"3815, WEST KIMBERLY RD",DAVENPORT,52806,82,Scott,1
930504,S34122500014,2016-08-25,75212,322,Koizumi Group,4694,Rina Mart LLC / Davenport,"3815, WEST KIMBERLY RD",DAVENPORT,52806,82,Scott,6
930505,S34122500015,2016-08-25,75224,322,Koizumi Group,4694,Rina Mart LLC / Davenport,"3815, WEST KIMBERLY RD",DAVENPORT,52806,82,Scott,6
930506,S34122500016,2016-08-25,36901,300,Katsuragi Company,4694,Rina Mart LLC / Davenport,"3815, WEST KIMBERLY RD",DAVENPORT,52806,82,Scott,6
